# Vaccine Listing Example

We show here how to fetch the information about ongoing clinical trials, and perform some simple searches on that information.

In [10]:
import requests
import pandas as pd
import os
import json

## Global Variable Initialization

We use a couple variables for all API calls. We initialize them here.

In [6]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
}

base_url = 'https://api.c3.ai/covid/api/1'

## Define data fetching function

This function fetches data varying the offset until all data from a given fetch query is gathered. Since the covid-19 Datalake public API currently limits fetches to at most 2000 entries, we will need to make multiple fetches until all data for that call has been gathered.

In [7]:
def fetch_data(url, headers=None, data=None):
    hasMore = True
    
    data['spec']['limit'] = -1
    data['spec']['offset'] = 0
    
    results = []
    
    while hasMore:
        r = requests.post(url,
                          headers=headers,
                          data=json.dumps(data))
        if r.status_code != 200:
            raise RuntimeError("Problem fetching data! {} ({})".format(r.text, r.request.body))
        
        res_json = json.loads(r.text)
        hasMore = res_json['hasMore']
        data['spec']['offset'] += res_json['count']
        
        results += res_json['objs']
    
    result_df = pd.DataFrame(results)
    result_df = result_df.set_index('id')
    return result_df

## Fetch Clinical trial data and have a peek

Let's fetch the clinical trial data and see what's there!

In [11]:
data = {
    'spec': {
    }
}

clinicaltrial_df = fetch_data(os.path.join(base_url, 'clinicaltrial', 'fetch'),
                              headers=headers,
                              data=data)

In [15]:
clinicaltrial_df

,url,pdf,country,location,startDate,trialStatus,design,blinding,covid19Status,patientSetting,outcome,treatmentType,meta,version,arms,severity,size,endDate
id,,,,,,,,,,,,,,,,,,
2020-000890-25_Paris_France,https://www.clinicaltrialsregister.eu/ctr-sear...,https://www.sciencedirect.com/science/article/...,France,{'id': 'Paris_France'},2020-03-01T00:00:00Z,Completed w. Results,Unspecified,Unspecified,Confirmed,Hospital,"Mortality, Hospitalization, Viral Load or Clea...",HCQ,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,NaN,NaN,NaN,NaN
2020-000919-69_Amsterdam_Netherlands,https://www.clinicaltrialsregister.eu/ctr-sear...,NaN,Netherlands,{'id': 'Amsterdam_Netherlands'},2020-03-01T00:00:00Z,Recruiting,Randomised,Double,Healthy (Exposed),Healthcare Workers,"Mortality, ICU Admission, Hospitalization, Vir...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,Healthy,NaN,NaN
2020-000936-23_Vienna_Austria,https://www.clinicaltrialsregister.eu/ctr-sear...,NaN,Austria,{'id': 'Vienna_Austria'},2020-05-01T00:00:00Z,Recruiting,Randomised,Open-Label,Confirmed,Hospital,NaN,LPV/r,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,4.0,NaN,3100.0,NaN
2020-000982-18_Oslo_Norway,https://www.clinicaltrialsregister.eu/ctr-sear...,NaN,Norway,{'id': 'Oslo_Norway'},2020-03-01T00:00:00Z,Recruiting,Randomised,Open-Label,Confirmed,"Hospital, ICU","Mortality, ICU Admission, Invasive Mechanical ...","HCQ, Remdesivir","{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,3.0,Severe,NaN,NaN
2020-001010-38_Oslo_Norway,https://www.clinicaltrialsregister.eu/ctr-sear...,NaN,Norway,{'id': 'Oslo_Norway'},2020-03-01T00:00:00Z,Recruiting,Randomised,Open-Label,Confirmed,Hospital,"Mortality, ICU Admission, Hospitalization, Cli...",HCQ,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,Mixed,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RPCEC00000306_Havana_Cuba,http://rpcec.sld.cu/en/trials/RPCEC00000306-En,NaN,Cuba,{'id': 'Havana_Cuba'},2020-04-01T00:00:00Z,Recruiting,Randomised,Unspecified,NaN,Healthy Exposed,"Viral Load or Clearance, Adverse Events, Serio...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,NaN,NaN,80.0,NaN
RPCEC00000307_Havana_Cuba,http://rpcec.sld.cu/en/trials/RPCEC00000307-En,NaN,Cuba,{'id': 'Havana_Cuba'},2020-04-01T00:00:00Z,Recruiting,Randomised,Unspecified,Confirmed,Unclear,Labor Absenteeism,IFN,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,NaN,Unclear,120.0,NaN
RPCEC00000308_Havana_Cuba,http://rpcec.sld.cu/en/trials/RPCEC00000308-En,NaN,Cuba,{'id': 'Havana_Cuba'},2020-03-01T00:00:00Z,Not Recruiting,Randomised,Unspecified,Healthy,Healthy Unexposed,NaN,IFN,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,NaN,None,30.0,NaN


How many clinical trial records did we get?

In [13]:
len(clinicaltrial_df)

2005

How many are for vaccines?

In [16]:
vaccine_df = clinicaltrial_df[clinicaltrial_df.treatmentType == 'Vaccine']

In [17]:
len(vaccine_df)

51

In [18]:
vaccine_df

,url,pdf,country,location,startDate,trialStatus,design,blinding,covid19Status,patientSetting,outcome,treatmentType,meta,version,arms,severity,size,endDate
id,,,,,,,,,,,,,,,,,,
2020-000919-69_Amsterdam_Netherlands,https://www.clinicaltrialsregister.eu/ctr-sear...,NaN,Netherlands,{'id': 'Amsterdam_Netherlands'},2020-03-01T00:00:00Z,Recruiting,Randomised,Double,Healthy (Exposed),Healthcare Workers,"Mortality, ICU Admission, Hospitalization, Vir...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,Healthy,NaN,NaN
2020-001072-15_London_UnitedKingdom,https://www.clinicaltrialsregister.eu/ctr-sear...,NaN,United Kingdom,{'id': 'London_UnitedKingdom'},2020-03-01T00:00:00Z,Recruiting,Randomised,Single,NaN,Healthy Unexposed,"Mortality, ICU Admission, Hospitalization, Vir...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,4.0,NaN,1112.0,NaN
2020-001591-15_Amsterdam_Netherlands,https://www.clinicaltrialsregister.eu/ctr-sear...,NaN,Netherlands,{'id': 'Amsterdam_Netherlands'},2020-04-01T00:00:00Z,Recruiting,Randomised,Single,NaN,Healthy Unexposed,"Mortality, ICU Admission, Hospitalization, Sec...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,NaN,NaN,NaN
2020-001678-31_Paris_France,https://www.clinicaltrialsregister.eu/ctr-sear...,NaN,France,{'id': 'Paris_France'},2020-04-01T00:00:00Z,Recruiting,Randomised,Single,Negative,"Healthy Unexposed, Healthy Exposed, Healthcare...","ICU Admission, Hospitalization, Viral Load or ...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,Healthy,NaN,NaN
2020-001783-28_Budapest_Hungary,https://www.clinicaltrialsregister.eu/ctr-sear...,NaN,Hungary,{'id': 'Budapest_Hungary'},2020-04-01T00:00:00Z,Recruiting,Randomised,Open-Label,Healthy,Healthcare Workers,"Mortality, ICU Admission, Labor Absenteeism, F...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,None,NaN,NaN
2020-001888-90_Copenhagen_Denmark,https://www.clinicaltrialsregister.eu/ctr-sear...,NaN,Denmark,{'id': 'Copenhagen_Denmark'},2020-04-01T00:00:00Z,Recruiting,Randomised,Single,Healthy,Healthcare Workers,"Mortality, ICU Admission, Labor Absenteeism, F...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,None,NaN,NaN
CHICTR2000030750_Guangdong_China,http://www.chictr.org.cn/showproj.aspx?proj=50928,NaN,China,{'id': 'Guangdong_China'},2020-03-01T00:00:00Z,Not Recruiting,Randomised,Unspecified,Confirmed,Unclear,"Non-invasive Ventilation, Invasive Mechanical ...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,4.0,Non-severe,120.0,2021-02-01T00:00:00Z
CHICTR2000031781_Jiangsu_China,http://www.chictr.org.cn/showproj.aspx?proj=52006,NaN,China,{'id': 'Jiangsu_China'},2020-04-01T00:00:00Z,Not Recruiting,Randomised,Double,NaN,Unclear,"Serious or Seconday Infections, Adverse Events...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,3.0,NaN,500.0,NaN
CHICTR2000032459_Henan_China,http://www.chictr.org.cn/showproj.aspx?proj=53003,NaN,China,{'id': 'Henan_China'},2020-04-01T00:00:00Z,Recruiting,Randomised,Double,Healthy,Healthy Exposed,NaN,Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,68.0,Healthy,288.0,NaN


How about Randomized, double-blind trials, with sample size > 1000?

In [24]:
rand_double_vacc_df = vaccine_df[(vaccine_df.design == 'Randomised') & (vaccine_df.blinding == 'Double') & (rand_double_vacc_df['size'] > 1000.)]

In [25]:
rand_double_vacc_df

,url,pdf,country,location,startDate,trialStatus,design,blinding,covid19Status,patientSetting,outcome,treatmentType,meta,version,arms,severity,size,endDate
id,,,,,,,,,,,,,,,,,,
NCT04327206_SouthAustralia_Australia,https://ClinicalTrials.gov/show/NCT04327206,NaN,Australia,{'id': 'SouthAustralia_Australia'},2020-03-01T00:00:00Z,Recruiting,Randomised,Double,Healthy (Exposed),Healthcare Workers,NaN,Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,NaN,10078.0,2020-10-01T00:00:00Z
NCT04327206_Victoria_Australia,https://ClinicalTrials.gov/show/NCT04327206,NaN,Australia,{'id': 'Victoria_Australia'},2020-03-01T00:00:00Z,Recruiting,Randomised,Double,Healthy (Exposed),Healthcare Workers,NaN,Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,NaN,10078.0,2020-10-01T00:00:00Z
NCT04327206_WesternAustralia_Australia,https://ClinicalTrials.gov/show/NCT04327206,NaN,Australia,{'id': 'WesternAustralia_Australia'},2020-03-01T00:00:00Z,Recruiting,Randomised,Double,Healthy (Exposed),Healthcare Workers,NaN,Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,NaN,10078.0,2020-10-01T00:00:00Z
NCT04348370_California_UnitedStates,https://ClinicalTrials.gov/show/NCT04348370,NaN,United States,{'id': 'California_UnitedStates'},2020-04-01T00:00:00Z,Recruiting,Randomised,Double,No,Healthcare Workers,"Clinical Improvement Score (Any), Viral Load o...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,Healthy,1800.0,2021-05-01T00:00:00Z
NCT04348370_Massachusetts_UnitedStates,https://ClinicalTrials.gov/show/NCT04348370,NaN,United States,{'id': 'Massachusetts_UnitedStates'},2020-04-01T00:00:00Z,Recruiting,Randomised,Double,No,Healthcare Workers,"Clinical Improvement Score (Any), Viral Load o...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,Healthy,1800.0,2021-05-01T00:00:00Z
NCT04348370_Texas_UnitedStates,https://ClinicalTrials.gov/show/NCT04348370,NaN,United States,{'id': 'Texas_UnitedStates'},2020-04-01T00:00:00Z,Recruiting,Randomised,Double,No,Healthcare Workers,"Clinical Improvement Score (Any), Viral Load o...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,Healthy,1800.0,2021-05-01T00:00:00Z
NCT04373291_Copenhagen_Denmark,https://clinicaltrials.gov/show/NCT04373291,NaN,Denmark,{'id': 'Copenhagen_Denmark'},2020-01-01T00:00:00Z,Not Recruiting,Randomised,Double,Healthy,Healthcare Workers,"Mortality, ICU Admission, Hospitalization, Hos...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,3.0,NaN,1500.0,NaN


We can see that there are no such trials for the general population:

In [26]:
rand_double_vacc_df.patientSetting.value_counts()

Healthcare Workers    7
Name: patientSetting, dtype: int64

What type of `patientSetting`s are there for vaccine trials?

In [27]:
vaccine_df.patientSetting.value_counts()

Healthy Unexposed                                         20
Healthcare Workers                                        19
Healthy Exposed                                            5
Hospital                                                   2
Unclear                                                    2
Healthy Exposed, Healthcare Workers                        1
Healthy Unexposed, Healthy Exposed, Healthcare Workers     1
Healthy Unexposed, Healthy Exposed, Outpatient             1
Name: patientSetting, dtype: int64

What trials involve 'Healthy Unexposed' people?

In [29]:
vaccine_df[vaccine_df.patientSetting.apply(lambda s: True if 'Healthy Unexposed' in s else False)]

,url,pdf,country,location,startDate,trialStatus,design,blinding,covid19Status,patientSetting,outcome,treatmentType,meta,version,arms,severity,size,endDate
id,,,,,,,,,,,,,,,,,,
2020-001072-15_London_UnitedKingdom,https://www.clinicaltrialsregister.eu/ctr-sear...,NaN,United Kingdom,{'id': 'London_UnitedKingdom'},2020-03-01T00:00:00Z,Recruiting,Randomised,Single,NaN,Healthy Unexposed,"Mortality, ICU Admission, Hospitalization, Vir...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,4.0,NaN,1112.0,NaN
2020-001591-15_Amsterdam_Netherlands,https://www.clinicaltrialsregister.eu/ctr-sear...,NaN,Netherlands,{'id': 'Amsterdam_Netherlands'},2020-04-01T00:00:00Z,Recruiting,Randomised,Single,NaN,Healthy Unexposed,"Mortality, ICU Admission, Hospitalization, Sec...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,NaN,NaN,NaN
2020-001678-31_Paris_France,https://www.clinicaltrialsregister.eu/ctr-sear...,NaN,France,{'id': 'Paris_France'},2020-04-01T00:00:00Z,Recruiting,Randomised,Single,Negative,"Healthy Unexposed, Healthy Exposed, Healthcare...","ICU Admission, Hospitalization, Viral Load or ...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,Healthy,NaN,NaN
NCT04283461_Georgia_UnitedStates,https://ClinicalTrials.gov/show/NCT04283461,NaN,United States,{'id': 'Georgia_UnitedStates'},2020-02-01T00:00:00Z,Recruiting,Non-Randomised,Open-Label,Healthy,Healthy Unexposed,"Mortality, ICU Admission, Hospital Discharge, ...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,Healthy,105.0,2021-09-01T00:00:00Z
NCT04283461_Maryland_UnitedStates,https://ClinicalTrials.gov/show/NCT04283461,NaN,United States,{'id': 'Maryland_UnitedStates'},2020-02-01T00:00:00Z,Recruiting,Non-Randomised,Open-Label,Healthy,Healthy Unexposed,"Mortality, ICU Admission, Hospital Discharge, ...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,Healthy,105.0,2021-09-01T00:00:00Z
NCT04283461_Washington_UnitedStates,https://ClinicalTrials.gov/show/NCT04283461,NaN,United States,{'id': 'Washington_UnitedStates'},2020-02-01T00:00:00Z,Recruiting,Non-Randomised,Open-Label,Healthy,Healthy Unexposed,"Mortality, ICU Admission, Hospital Discharge, ...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,Healthy,105.0,2021-09-01T00:00:00Z
NCT04299724_Guangdong_China,https://ClinicalTrials.gov/show/NCT04299724,NaN,China,{'id': 'Guangdong_China'},2020-03-01T00:00:00Z,Recruiting,Single-Arm,Open-Label,Confirmed And Healthy,"Healthy Unexposed, Healthy Exposed, Outpatient","Mortality, Clinical Improvement Score (Any), I...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,1.0,NaN,100.0,2023-07-01T00:00:00Z
NCT04313127_Hubei_China,https://ClinicalTrials.gov/show/NCT04313127,NaN,China,{'id': 'Hubei_China'},2020-03-01T00:00:00Z,Not Recruiting,Non-Randomised,Open-Label,Healthy,Healthy Unexposed,"Treatment-emergent Adverse Events, Serious Adv...",Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,2.0,Healthy,108.0,2020-12-01T00:00:00Z
NCT04324606_Bristol_UnitedKingdom,https://ClinicalTrials.gov/show/NCT04324606,NaN,United Kingdom,{'id': 'Bristol_UnitedKingdom'},2020-03-01T00:00:00Z,Not Recruiting,Randomised,Single,Healthy,Healthy Unexposed,NaN,Vaccine,"{'tenantTagId': 4, 'tenant': 'covid', 'tag': '...",1,4.0,Healthy,1090.0,2021-05-01T00:00:00Z
